In [20]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

from keras.applications.vgg16 import VGG16 # load VGG16 model from keras
from keras.preprocessing.image import load_img # Using images library
from keras.preprocessing.image import img_to_array # # convert image pixels to numpy for specific manipulations
from keras.applications.vgg16 import preprocess_input # to prepare for new input
from keras.applications.vgg16 import decode_predictions # for reporting probabilities

#For Keras
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, save_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

In [21]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
# prepare an iterators for each dataset
train_it = train_datagen.flow_from_directory('data/images/train',
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     batch_size=32,
                                     class_mode='categorical',
                                     shuffle=True)

validate_it = train_datagen.flow_from_directory('data/images/validate',
                                     target_size=(224,224),
                                     color_mode='rgb',
                                     batch_size=32,
                                     class_mode='categorical',
                                     shuffle=True)

train_batchX, train_batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (train_batchX.shape, train_batchX.min(), train_batchX.max()))
validate_batchX, validate_batchy = validate_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (validate_batchX.shape, validate_batchX.min(), validate_batchX.max()))

Found 225 images belonging to 5 classes.
Found 48 images belonging to 5 classes.
Batch shape=(32, 224, 224, 3), min=-123.680, max=151.061
Batch shape=(32, 224, 224, 3), min=-123.680, max=151.061


In [22]:
# classifying only weapons 
vgg16_model = VGG16(weights="imagenet")

In [23]:
# DO NOT RUN IF YOU ARE DOING CPU
#pysical_devices = tf.config.list_physical_devices('GPU') 

#try: 
    #  tf.config.experimental.set_memory_growth(physical_devices[0], True)
#except: 
    # Invalid device or cannot modify virtual devices once initialized. 
    #  pass 

In [24]:
# Check devices on computer!
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [25]:
seq_model = Sequential()

# iterate all layers in vgg16 to sequential for custom modification
for layer in vgg16_model.layers:
    seq_model.add(layer)
    
seq_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [26]:
# 1000 categories, pop off output layer to work only specific categories
seq_model._layers.pop()

In [27]:
# freeze weights to prevent constantly updating weights
for layer in seq_model._layers:
    layer.trainable = True
    # SET TO FALSE IF YOU DO NOT WANNA TRAIN THE MODEL!

In [28]:
seq_model.add(Dense(5, activation='softmax')) # 5 categories 

# seq_model.summary()

In [29]:
seq_model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train = train_it.n//train_it.batch_size #np.ceil((train_it.n//train_it.batch_size)/2)
history = seq_model.fit_generator(generator=train_it,
                                  validation_data=validate_it,
                                  validation_freq=1,
                                  steps_per_epoch=step_size_train,
                                  shuffle=True,
                                  epochs=5)

Epoch 1/5


ResourceExhaustedError:  OOM when allocating tensor with shape[32,128,112,112] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node block2_conv2_3/convolution (defined at C:\Users\xris1\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_18899]

Function call stack:
keras_scratch_graph


In [5]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [10]:
# TO PREDICT PROPERLY YOU NEED TO FEED IT AN IMAGE THAT IS NOT IN THE VALIDATION OR TEST!
prediction = seq_model.predict(validate_batchX)

# convert the probabilities to class labels
#label = decode_predictions(pred)

for row in range(len(prediction)):
    biggest = np.argmax(prediction[row])
    for val in range(len(prediction[row])):
        if(val == biggest):
            prediction[row][val] = 1.
        else:
            prediction[row][val] = 0.

# retrieve the most likely result, e.g. highest probability
test_prediction = prediction == validate_batchy

In [11]:
# HOW MANY OF THE TESTS WERE WRONG
print((np.sum(test_prediction % 2 == 0))/2)

24.0
